In [ ]:
%load_ext aiida
%aiida
from IPython.display import display, clear_output
import ipywidgets as ipw
import widgets
import utils
import aiida_utils
import json
import shutil

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
slabs_options = [object_key for object_key, object_info in CONFIG["objects"].items() if object_info["object_type"] == "slab"]
slabs_options.insert(0, "No material")
material_selection_radio_button = utils.Radiobuttons(
    description = '', options = slabs_options, 
    disabled = False, layout = ipw.Layout(width = '300px'), 
    style = {'description_width': "100px"}
)

simulation_material_dropdown = utils.Dropdown(
    description = "Simulation material", disabled = False, 
    layout = ipw.Layout(width = "300px"), 
    options = ["Select an option...", "Molecule and slab", "Reaction product"],
    value = "Select an option...",
    style = {'description_width': "150px"}                
)
simulation_material_properties_output = ipw.Output()

material_selector = widgets.MaterialSelectionWidget()

experiment_selector = widgets.ExperimentSelectionWidget()
experiment_selector.load_dropdown_box()

molecule_selector_list = widgets.MultipleSelectorWidget("molecule")
molecule_selector_list_output = ipw.Output()

add_molecule_button = utils.Button(
    description = 'Add molecule', disabled = False, button_style = '', 
    tooltip = 'Add molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_molecule_button = utils.Button(
    description = 'Remove molecule', disabled = False, button_style = '', 
    tooltip = 'Remove molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_molecule_buttons_hbox = ipw.HBox([add_molecule_button, remove_molecule_button])

product_selector_list = widgets.MultipleSelectorWidget("product")
product_selector_list_output = ipw.Output()

add_product_button = utils.Button(
    description = 'Add product', disabled = False, button_style = '', 
    tooltip = 'Add product', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_product_button = utils.Button(
    description = 'Remove product', disabled = False, button_style = '', 
    tooltip = 'Remove product', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_product_buttons_hbox = ipw.HBox([add_product_button, remove_product_button])

simulation_selector = widgets.SimulationSelectionWidget()
simulation_selector.load_dropdown_box()

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def select_material_radio_change(change):
    material_type = material_selection_radio_button.value
    if material_type == "No material":
        with material_selector:
            clear_output()
            return
    
    material_selector.details_textbox.value = ''
    material_types = {}
    for object_key, object_info in CONFIG["objects"].items():
        if object_info["object_type"] == "slab":
            material_types[object_key] = (object_info["openbis_object_type"], object_info["placeholder"])
            
    with material_selector:
        clear_output()
        display_list = [
            material_selector.dropdown_boxes,
            ipw.HBox([material_selector.details_textbox, material_selector.image_box])
        ]
        material_class, placeholder = material_types.get(material_type)
        material_selector.load_dropdown_box(material_class, placeholder)
        display(ipw.VBox(display_list))

# Function to handle changes in the materials dropdown
def load_material_metadata(change):
    if material_selector.dropdown_boxes.children[0].value == -1:
        material_selector.details_textbox.value = ''
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])
        return
    
    # Get selected object properties information from config file
    selected_object = material_selection_radio_button.value
    selected_object_properties = CONFIG["objects"][selected_object]["properties"]
    
    # Get material object information and dataset
    material_object = OPENBIS_SESSION.get_object(material_selector.dropdown_boxes.children[0].value)
    material_dataset = material_object.get_datasets()[0]
    
    # Get the object image preview
    if material_dataset:
        material_dataset.download(destination="images")
        material_selector.image_box.value = utils.read_file(f"images/{material_dataset.permId}/{material_dataset.file_list[0]}")
        # Erase file after downloading it
        shutil.rmtree(f"images/{material_dataset.permId}")
    else:
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])

    # Make a string with the property values of the object
    material_metadata = material_object.props.all()
    material_metadata_string = ""
    for prop_key in selected_object_properties:
        prop_title = CONFIG["properties"][prop_key]["title"]
        if CONFIG["properties"][prop_key]["property_type"] == "QUANTITY_VALUE":
            value = material_metadata.get(prop_key)
            if value:
                prop_dict = json.loads(value)
                material_metadata_string += f"{prop_title}: {prop_dict['value']} {prop_dict['unit']}\n"
            else:
                material_metadata_string += f"{prop_title}: {value}\n"
        else:
            material_metadata_string += f"{prop_title}: {material_metadata.get(prop_key)}\n"

    material_selector.details_textbox.value = material_metadata_string

def add_molecule_widget(b):
    molecule_selector_list.add_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def remove_molecule_widget(b):
    molecule_selector_list.remove_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def add_product_widget(b):
    product_selector_list.add_selector()
    with product_selector_list_output:
        clear_output()
        display(product_selector_list)

def remove_product_widget(b):
    product_selector_list.remove_selector()
    with product_selector_list_output:
        clear_output()
        display(product_selector_list)

def find_atomistic_model_perm_id(data):
    if isinstance(data, dict):
        # Check if the current object is of type ATOMISTIC_MODEL and is at the deepest level
        if data.get('type') == 'ATOMISTIC_MODEL' and (not data.get('has_part') or not isinstance(data['has_part'], list)):
            return data['perm_id']
        # Recurse into 'has_part' if it exists
        if 'has_part' in data:
            for item in data['has_part']:
                result = find_atomistic_model_perm_id(item)
                if result:
                    return result
    
    elif isinstance(data, list):
        # Recurse into list elements
        for item in data:
            result = find_atomistic_model_perm_id(item)
            if result:
                return result
    
    return None
      
def create_simulation_openbis(b):
    if experiment_selector.dropdown.value == -1:
        display(utils.Javascript(data = "alert('Select an experiment.')"))
        return
    else:
        last_export = aiida_utils.export_workchain(
            OPENBIS_SESSION, 
            experiment_selector.dropdown.value, 
            simulation_selector.dropdown.value
        )
        
        if last_export:
            last_export_metadata = utils.get_parent_child_relationships_nested(OPENBIS_SESSION, last_export)
            first_atomistic_model_permid = find_atomistic_model_perm_id(last_export_metadata)
            first_atomistic_model = utils.get_openbis_objects(OPENBIS_SESSION, identifier = first_atomistic_model_permid)[0]
            if first_atomistic_model.parents is None:
                first_atomistic_model_parents = []
                if molecule_selector_list.selectors:
                    for molecule_selector in molecule_selector_list.selectors:
                        if molecule_selector.dropdown.value != -1:
                            first_atomistic_model_parents.append(molecule_selector.dropdown.value)
                if material_selector.dropdown.value != -1:
                    first_atomistic_model_parents.append(material_selector.dropdown.value)
                if product_selector_list.selectors:
                    for product_selector in product_selector_list.selectors:
                        if product_selector.dropdown.value != -1:
                            first_atomistic_model_parents.append(product_selector.dropdown.value)
                first_atomistic_model.set_parents(first_atomistic_model_parents)
                first_atomistic_model.save()

# Send simulation to openBIS

## Select experiment

In [ ]:
display(experiment_selector)

## Select molecule

In [ ]:
display(molecule_selector_list_output)
with molecule_selector_list_output:
    display(molecule_selector_list)  
display(add_remove_molecule_buttons_hbox)

## Select reaction product

In [ ]:
display(product_selector_list_output)
with product_selector_list_output:
    display(product_selector_list)  
display(add_remove_product_buttons_hbox)

## Select slab

In [ ]:
display(material_selection_radio_button)
display(material_selector)

## Select simulation

In [ ]:
display(simulation_selector)

## Save simulation workchain

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
material_selection_radio_button.observe(select_material_radio_change, names='value')
material_selector.dropdown.observe(load_material_metadata, names = 'value')
add_molecule_button.on_click(add_molecule_widget)
remove_molecule_button.on_click(remove_molecule_widget)
add_product_button.on_click(add_product_widget)
remove_product_button.on_click(remove_product_widget)
create_button.on_click(create_simulation_openbis)
quit_button.on_click(close_notebook)

In [41]:
import base64

# Path to the PNG file
file_path = '/home/jovyan/aiida-openbis/Notebooks/connection_to_openbis/ase_geo.png'

# Convert PNG to bytes string
with open(file_path, 'rb') as file:
    binary_data = file.read()
    bytes_string = base64.b64encode(binary_data).decode('utf-8')

print(bytes_string)

...


In [ ]:
# CONFIG = utils.read_json("config.json")
# CONFIG_ELN = utils.read_json("eln_config.json")
# OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
# utils.create_openbis_object(OPENBIS_SESSION, type = "2D_MEASUREMENT",
#                             props = {"bias_voltages": [json.dumps({"has_value": float(i)*1.602176634e-19, "has_unit": "unit:V"}) for i in ['-2.0', '0.0']]},
#                             collection = "/ATOMISTIC_SIMULATIONS/EXAMPLE_PROJECT/EXAMPLE_PROJECT_EXP_8")